In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import math
import requests

In [40]:
sr = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/soldresults10yr.csv")
hrn = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/hrnmaster606.csv")
ped = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/pedmaster606.csv")
#df = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/sdperformance528.csv")

In [41]:
unwanted = hrn.columns[hrn.columns.str.startswith('Unnamed:')]
hrn.drop(unwanted, axis=1, inplace=True)
print(hrn.shape)
hrn = hrn.drop_duplicates(keep='first')
print(hrn.shape)

unwanted = ped.columns[ped.columns.str.startswith('Unnamed:')]
ped.drop(unwanted, axis=1, inplace=True)
print(ped.shape)
ped = ped.drop_duplicates(keep='first')
print(ped.shape)

(471251, 12)
(468556, 12)
(100689, 15)
(100689, 15)


In [42]:
sirelist = list(sr['Siref'].unique())
damlist = list(sr['Damf'].unique())
damsirelist = list(ped['Name'].unique())

In [43]:
hrn['Date']
substring = 'R'
dde = list(hrn['Date'])
dde
datelist = list()
for i in list(hrn['Date']):
    if i is np.nan:
        datelist.append(np.nan)    
    elif substring in i:
        datelist.append(i.lstrip().split('-')[0])
    else:
        datelist.append(i)

hrn['Datecol'] = datelist

hrn['Datecol'] = pd.to_datetime(hrn["Datecol"])
sr['OBSDate'] = pd.to_datetime(sr['OBSDate'])

In [46]:
looplist = sr[['OBSDate', 'Siref', 'Damf']].apply(tuple, axis=1).tolist()
looplist[0]

(Timestamp('2011-01-01 00:00:00'), 'Unbridled', 'Nannerl')

In [ ]:
heads = ['Date', 'Sire', 's_firstrace', 's_nraces', 's_nwins', 's_g1', 's_g2', 's_g3', 's_firstprogenyrace', 's_pnraces', 's_pnwins','s_pg1','s_pg2','s_pg3',
        'Dam', 'd_firstrace', 'd_nraces', 'd_nwins', 'd_g1', 'd_g2', 'd_g3', 'd_firstprogenyrace', 'd_pnraces', 'd_pnwins','d_pg1','d_pg2','d_pg3',
        'Damsire', 'ds_firstrace', 'ds_nraces', 'ds_nwins', 'ds_g1', 'ds_g2', 'ds_g3', 'ds_firstprogenyrace', 'ds_pnraces', 'ds_pnwins','ds_pg1','ds_pg2','ds_pg3']
headp = ['Horse', 'firstprogenyrace', 'pnraces', 'pnwins','pg1','pg2','pg3']
df = pd.DataFrame(columns = heads) # creates master dataframe 
nullprog = list([np.nan, 0, 0, 0, 0, 0])
#for p in ped['Name']:
#    p = p.replace('_','')

for m in looplist:
    
    stats2 = list()
    stats2.append(m[0])
    adate = hrn[hrn['Datecol'] < (m[0] + pd.DateOffset(months = 1))]
#    adate = hrn[hrn['Datecol'] < m[0]]
    sire = m[1]
    dam = m[2]
    
    #            SIRE STATISTICS
    stats2.append(sire)
        
    srresult = adate[adate['Horse'] == sire].dropna(subset=['Fin'])
    stats2.append(min(srresult['Datecol'], default=np.nan))
    
    srresult2 = srresult.dropna(subset=['Fin'])
    stats2.append(srresult2.shape[0])
            
    srresult3 = srresult2[srresult2['Fin'] == '1']
    srresult4 = srresult2[srresult2['Fin'] == '1st']
    srresult4 = srresult4.append(srresult3)
    stats2.append(srresult4.shape[0])
                
    srresult4['Grade'] = srresult4['Race'].str[-2:]
    stats2.append(srresult4[srresult4['Grade'] == 'G1'].shape[0])
    stats2.append(srresult4[srresult4['Grade'] == 'G2'].shape[0])
    stats2.append(srresult4[srresult4['Grade'] == 'G3'].shape[0])
    
    spro = ped[ped['S'] == sire]
    sprolist = list(spro['Name'].unique())
    
#    if len(sprolist) == 0:
#        for i in nullprog:
#            stats.append(i)
    
    df1 = pd.DataFrame(columns=headp)
    for px in sprolist:
        stats = list()
        stats.append(px)
        
        presult = adate[adate['Horse'] == px].dropna(subset=['Fin'])       
        
#        mindate = np.nanmin(presult['Datecol'])
        stats.append(min(presult['Datecol'], default=np.nan))
        
        presult2 = presult[presult['Fin'] == '1']
        presult3 = presult[presult['Fin'] == '1st']
        presult3 = presult3.append(presult2)
        stats.append(presult.shape[0])
        stats.append(presult3.shape[0])
        
        presult3['Grade'] = presult3['Race'].str[-2:]
        
        stats.append(presult3[presult3['Grade'] == 'G1'].shape[0])
        stats.append(presult3[presult3['Grade'] == 'G2'].shape[0])     
        stats.append(presult3[presult3['Grade'] == 'G3'].shape[0])
        
        df1_length = len(df1)
        df1.loc[df1_length] = stats
    
    df2 = df1.dropna(subset=['firstprogenyrace'])
#    md = min(df1['firstprogenyrace'])
    stats2.append(min(df2['firstprogenyrace'], default=np.nan))
                
    stats2.append(sum(df1['pnraces']))
    stats2.append(sum(df1['pnwins']))
    stats2.append(sum(df1['pg1']))
    stats2.append(sum(df1['pg2']))
    stats2.append(sum(df1['pg3']))
    
    #                DAM STATISTICS
    
    stats2.append(dam)
        
    drresult = adate[adate['Horse'] == dam].dropna(subset=['Fin'])
    stats2.append(min(drresult['Datecol'], default=np.nan))
    
    drresult2 = drresult.dropna(subset=['Fin'])
    stats2.append(drresult2.shape[0])
            
    drresult3 = drresult2[drresult2['Fin'] == '1']
    drresult4 = drresult2[drresult2['Fin'] == '1st']
    drresult4 = drresult4.append(drresult3)
    stats2.append(drresult4.shape[0])
                
    drresult4['Grade'] = drresult4['Race'].str[-2:]
    stats2.append(drresult4[drresult4['Grade'] == 'G1'].shape[0])
    stats2.append(drresult4[drresult4['Grade'] == 'G2'].shape[0])
    stats2.append(drresult4[drresult4['Grade'] == 'G3'].shape[0])
    
    dpro = ped[ped['D'] == dam]
    dprolist = list(dpro['Name'].unique())
    
#    if len(dprolist) == 0:
#        for i in nullprog:
#            stats.append(i)
    
    df1 = pd.DataFrame(columns=headp)
    for px in dprolist:
        stats = list()
        stats.append(px)
        
        presult = adate[adate['Horse'] == px].dropna(subset=['Fin'])       
        
#        mindate = np.nanmin(presult['Datecol'])
        stats.append(min(presult['Datecol'], default=np.nan))
        
        presult2 = presult[presult['Fin'] == '1']
        presult3 = presult[presult['Fin'] == '1st']
        presult3 = presult3.append(presult2)
        stats.append(presult.shape[0])
        stats.append(presult3.shape[0])
        
        presult3['Grade'] = presult3['Race'].str[-2:]
        
        stats.append(presult3[presult3['Grade'] == 'G1'].shape[0])
        stats.append(presult3[presult3['Grade'] == 'G2'].shape[0])     
        stats.append(presult3[presult3['Grade'] == 'G3'].shape[0])
        
        df1_length = len(df1)
        df1.loc[df1_length] = stats
    
    df2 = df1.dropna(subset=['firstprogenyrace'])
#    md = min(df1['firstprogenyrace'])
    stats2.append(min(df2['firstprogenyrace'], default=np.nan))
                
    stats2.append(sum(df1['pnraces']))
    stats2.append(sum(df1['pnwins']))
    stats2.append(sum(df1['pg1']))
    stats2.append(sum(df1['pg2']))
    stats2.append(sum(df1['pg3']))
    
    #         DAMSIRE STATISTICS
    
    damped = ped[ped['Name'] == dam]
    if damped.shape[0] == 0:
        damsire = np.nan
    else:
        damsire = damped['S'].iloc[0]
    stats2.append(damsire)
        
    drresult = adate[adate['Horse'] == damsire].dropna(subset=['Fin'])
    stats2.append(min(drresult['Datecol'], default=np.nan))
    
    drresult2 = drresult.dropna(subset=['Fin'])
    stats2.append(drresult2.shape[0])
            
    drresult3 = drresult2[drresult2['Fin'] == '1']
    drresult4 = drresult2[drresult2['Fin'] == '1st']
    drresult4 = drresult4.append(drresult3)
    stats2.append(drresult4.shape[0])
                
    drresult4['Grade'] = drresult4['Race'].str[-2:]
    stats2.append(drresult4[drresult4['Grade'] == 'G1'].shape[0])
    stats2.append(drresult4[drresult4['Grade'] == 'G2'].shape[0])
    stats2.append(drresult4[drresult4['Grade'] == 'G3'].shape[0])
    
    dpro = ped[ped['S'] == damsire]
    dprolist = list(dpro['Name'].unique())
    
#    if len(dprolist) == 0:
#        for i in nullprog:
#            stats.append(i)
    
    df1 = pd.DataFrame(columns=headp)
    for px in dprolist:
        stats = list()
        stats.append(px)
        
        presult = adate[adate['Horse'] == px].dropna(subset=['Fin'])       
        
#        mindate = np.nanmin(presult['Datecol'])
        stats.append(min(presult['Datecol'], default=np.nan))
        
        presult2 = presult[presult['Fin'] == '1']
        presult3 = presult[presult['Fin'] == '1st']
        presult3 = presult3.append(presult2)
        stats.append(presult.shape[0])
        stats.append(presult3.shape[0])
        
        presult3['Grade'] = presult3['Race'].str[-2:]
        
        stats.append(presult3[presult3['Grade'] == 'G1'].shape[0])
        stats.append(presult3[presult3['Grade'] == 'G2'].shape[0])     
        stats.append(presult3[presult3['Grade'] == 'G3'].shape[0])
        
        df1_length = len(df1)
        df1.loc[df1_length] = stats
    
    df2 = df1.dropna(subset=['firstprogenyrace'])
#    md = min(df1['firstprogenyrace'])
    stats2.append(min(df2['firstprogenyrace'], default=np.nan))
                
    stats2.append(sum(df1['pnraces']))
    stats2.append(sum(df1['pnwins']))
    stats2.append(sum(df1['pg1']))
    stats2.append(sum(df1['pg2']))
    stats2.append(sum(df1['pg3']))
    
    df_length = len(df)
    df.loc[df_length] = stats2
                  
    df.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\sdperformance606.csv')
    
df
        

In [26]:
df.columns

Index(['Date', 'Sire', 's_firstrace', 's_nraces', 's_nwins', 's_g1', 's_g2',
       's_g3', 's_firstprogenyrace', 's_pnraces', 's_pnwins', 's_pg1', 's_pg2',
       's_pg3', 'Dam', 'd_firstrace', 'd_nraces', 'd_nwins', 'd_g1', 'd_g2',
       'd_g3', 'd_firstprogenyrace', 'd_pnraces', 'd_pnwins', 'd_pg1', 'd_pg2',
       'd_pg3Damsire', 'ds_firstrace', 'ds_nraces', 'ds_nwins', 'ds_g1',
       'ds_g2', 'ds_g3', 'ds_firstprogenyrace', 'ds_pnraces', 'ds_pnwins',
       'ds_pg1', 'ds_pg2', 'ds_pg3'],
      dtype='object')

In [25]:
len(stats2

40

In [140]:
sort = df.sort_values('pg1', ascending = False)
sort.head(30)

,Horse,firstrace,nraces,nwins,g1,g2,g3,firstprogenyrace,pnraces,pnwins,pg1,pg2,pg3
277,Tapit,2003-10-19,6,3,1,0,1,2006-11-13,1203,238,26,20,16
88,Smart Strike,1996-08-25,2,1,1,0,0,2002-05-18,873,137,23,11,10
5,Unbridled's Song,1995-08-26,13,5,2,1,0,2000-09-04,903,172,19,14,11
562,Storm Cat,1985-10-10,2,1,1,0,0,1993-11-06,312,56,18,9,3
121,Tale of the Cat,1997-08-03,5,1,0,1,0,2003-02-23,801,140,18,12,9
251,Bernardini,2006-01-07,8,6,3,1,1,2010-07-09,739,129,15,10,8
139,Gone West,1987-06-06,3,0,0,0,0,1992-11-12,291,58,14,10,5
753,Kingmambo,1992-07-24,12,5,3,0,0,1998-03-22,99,24,13,5,0
160,Gulch,1986-11-01,10,1,1,0,0,1994-09-16,144,32,13,5,1
224,Deputy Minister,NaT,0,0,0,0,0,1993-07-29,85,25,12,7,1


In [150]:
xcross = sr.sort_values('Pricef', ascending=False)
xcross[['Damf','Siref','Pricef','OBSDate']].head(30)

,Damf,Siref,Pricef,OBSDate
10600,NaN,NaN,53291900.0,2015-04-01
11737,Moonbow,Tiznow,2450000.0,2017-04-01
22848,Artemis Agrotera,Tapit,2000000.0,2019-03-01
5759,Artemis Agrotera,Tapit,2000000.0,2019-03-01
10181,Silver Wolf,Tapit,1900000.0,2015-04-01
4584,Mini Sermon,Smart Strike,1800000.0,2013-03-01
5607,Azalea Belle,Congrats,1700000.0,2017-03-01
5436,Glamorista,Smart Strike,1700000.0,2016-03-01
4766,Rebridled Dreams,Giant's Causeway,1600000.0,2014-03-01
4770,Rhumb Line,Tapit,1600000.0,2014-03-01


In [152]:
df[df['Horse'] == 'Satareh']

,Horse,firstrace,nraces,nwins,g1,g2,g3,firstprogenyrace,pnraces,pnwins,pg1,pg2,pg3


In [61]:
df.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\sdprecall.csv')

In [146]:
proglist[proglist['Name'] == 'Moonbow']

,Unnamed: 0,Name,Progeny_raw
12417,25826,Moonbow,Conquistador (USA)\nGreen Moon Rising (USA)\nM...


In [148]:
hrn[hrn['Horse'] == 'Conquistador']

,Date,Horse,Fin,Tr,Dist.,Sf.,Race,Cnd,1st,2nd,3rd,Time,Datecol


In [149]:
conq = pd.read_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\hrnmaster514.csv')
conq[conq['Horse'] == 'Conquistador']

,Unnamed: 0,Date,Horse,Fin,Tr,Dist.,Sf.,Race,Cnd,1st,2nd,3rd,Time
183912,183912,03/14/20-R3,Conquistador,1st,Aqu,6 f,D,Mcl,3+,Conquistador,Mr. Mike,Tipazo,1:13.53
183913,183913,01/26/20-R1,Conquistador,7th,Aqu,7 f,D,MSW,4+,Flying Curlin,Fast Break,Spice Road,1:25.13
183914,183914,01/05/20-R6,Conquistador,3rd,Aqu,6 f,D,MSW,4+,Hip Hip Jorge,New Frontier,Conquistador,1:11.60
183915,183915,01/31/19-R8,Conquistador,10th,FG,5 1/2 f,T,MSW,3+,Vogt,Morning Social,Red Mule,1:04.78
183916,183916,10/29/17-R11,Conquistador,3rd,CD,1 mile,D,MSW,2,Combatant,Combined,Conquistador,1:37.33
183917,183917,10/07/17-R3,Conquistador,2nd,Kee,6 f,D,MSW,2,Rubus,Conquistador,Tricks To Doo,1:10.53


In [17]:
damped = ped[ped['Name'] == 'Practical Joke']
damped['S'].iloc[0]


'Into Mischief'

In [28]:
damped

,Name,S,SS,SSS,SSD,SD,SDS,SDD,D,DS,DSS,DSD,DD,DDS,DDD


In [31]:
m

(Timestamp('2012-01-01 00:00:00'), nan, nan)

In [30]:
looplist

[(Timestamp('2011-01-01 00:00:00'), 'Unbridled', 'Nannerl'),
 (Timestamp('2011-01-01 00:00:00'), 'Wildcat Heir', 'Note to Mimi'),
 (Timestamp('2011-01-01 00:00:00'), 'Private Vow', 'Parkdale'),
 (Timestamp('2011-01-01 00:00:00'), 'Matty G', 'Party Ribbon'),
 (Timestamp('2011-01-01 00:00:00'), 'With Distinction', 'Pearlwood'),
 (Timestamp('2011-01-01 00:00:00'), "Unbridled's Song", 'Pheroz Fantasy (NZ)'),
 (Timestamp('2011-01-01 00:00:00'), 'Wildcat Heir', 'Pretty Indian'),
 (Timestamp('2011-01-01 00:00:00'), "Chief's Crown", 'Princess Caveat'),
 (Timestamp('2011-01-01 00:00:00'), 'Alphabet Soup', 'Private Fun'),
 (Timestamp('2011-01-01 00:00:00'), 'Shaniko', 'Private Prom Party'),
 (Timestamp('2011-01-01 00:00:00'), 'Siphon (BRZ)', 'Really Quick'),
 (Timestamp('2011-01-01 00:00:00'), "Yes It's True", 'Rebridled Secret'),
 (Timestamp('2011-01-01 00:00:00'), 'Posse', 'Red Cause'),
 (Timestamp('2011-01-01 00:00:00'), 'Alphabet Soup', 'Retroesque'),
 (Timestamp('2011-01-01 00:00:00'), 'Son

In [32]:
looplist.index(m)

630

In [34]:
looplist[630:]

[(Timestamp('2012-01-01 00:00:00'), nan, nan),
 (Timestamp('2012-01-01 00:00:00'), nan, nan),
 (Timestamp('2013-01-01 00:00:00'), 'Desert Party', 'Zakcat'),
 (Timestamp('2013-01-01 00:00:00'), 'Neko Bay', 'Zelmira'),
 (Timestamp('2013-01-01 00:00:00'), 'Divine Park', 'Al Max Diner'),
 (Timestamp('2013-01-01 00:00:00'), 'Leroidesanimaux (BRZ)', 'A. M. Panicus'),
 (Timestamp('2013-01-01 00:00:00'), 'A.P. Indy', 'Ampulla'),
 (Timestamp('2013-01-01 00:00:00'), 'Circular Quay', 'Angel of Bataan'),
 (Timestamp('2013-01-01 00:00:00'), 'Mineshaft', 'Anklet'),
 (Timestamp('2013-01-01 00:00:00'), 'Yes Its True', 'Ayala Strand'),
 (Timestamp('2013-01-01 00:00:00'), 'Majestic Warrior', 'Baba Gonzo'),
 (Timestamp('2013-01-01 00:00:00'), 'Kittens Joy', 'Bella Trella'),
 (Timestamp('2013-01-01 00:00:00'), 'Temple City', 'Brightest Future'),
 (Timestamp('2013-01-01 00:00:00'), 'Empire Maker', 'Brink'),
 (Timestamp('2013-01-01 00:00:00'), 'Holy Bull', 'Brush With Reality'),
 (Timestamp('2013-01-01 00:0

In [35]:
damped

,Name,S,SS,SSS,SSD,SD,SDS,SDD,D,DS,DSS,DSD,DD,DDS,DDD


In [39]:
hrn[hrn['Horse'] == 'Cherokee Run']

,Date,Horse,Fin,Tr,Dist.,Sf.,Race,Cnd,1st,2nd,3rd,Time,Datecol
1763,11/05/94-R4,Cherokee Run,1,CD,6 f,D,Breeders' Cup Sprint-G1,3+,Cherokee Run,Soviet Problem,Cardmania,1:09.40,1994-11-05
1764,05/30/94-R8,Cherokee Run,2,Bel,1 mile,D,Metropolitan -G1,3+,Holy Bull,Cherokee Run,Devil His Due,1:33.80,1994-05-30
1766,08/21/93-R7,Cherokee Run,7,Sar,1 1/4 m,D,Travers Stakes-G1,3+,Sea Hero,Kissin Kris,Miner's Mark,2:01.95,1993-08-21
1768,08/01/93-R11,Cherokee Run,4,Mth,1 1/8 m,D,Haskell Stakes-G1,3+,Kissin Kris,Storm Tower,Dry Bean,1:49.58,1993-08-01
1770,06/05/93-R9,Cherokee Run,6,Bel,1 1/2m,D,Belmont Stakes-G1,3,Colonial Affair,Kissin Kris,Wild Gale,2:29.80,1993-06-05
1772,05/15/93-R10,Cherokee Run,2,Pim,1 3/16 m,D,Preakness Stakes-G1,3,Prairie Bayou,Cherokee Run,El Bakan,1:56.60,1993-05-15
